In [ ]:
# !pip uninstall torchtext
!pip install torchtext


In [ ]:
# !pip uninstall portalocker
!pip install portalocker --no-cache-dir


In [ ]:
# !pip uninstall accelerate
!pip install accelerate -U --no-cache-dir


In [2]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

OSError: /mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN2at4_ops5zeros4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEE

In [ ]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')


import random
random.seed(6)
print('FFF', train_iter)

train_list = list(train_iter)
print('KKK')
test_list = list(test_iter)
print("AAA")
train_lists_small = random.sample(train_list, 1000)
print("BBB")
test_lists_small = random.sample(test_list, 1000)
print("CCC")

print(train_lists_small[0])
print(test_lists_small[0])

FFF ShardingFilterIterDataPipe
KKK
AAA
BBB
CCC
(2, "I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.")
(1, 'This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and pur

## 레이블 인코딩

- 텍스트와 레이블을 Tuple pair
- 긍정 2
- 부정 1
- 2를 -> 1로
- 1을 -> 2로

In [ ]:
train_texts = []
train_labels = []

# train_lists_small에 담긴 튜플 쌍 원소를 변수명 label과 text를 부여하여 순서대로 추출

for label, text in train_lists_small:
    # IMDB 데이터의 기존 레이블 2를 1로 변경 기존 레이블 1을 0으로 변경
    train_labels.append(1 if label==2 else 0)
    train_texts.append(text)

test_texts = []
test_labels = []

for label, text in test_lists_small:
    test_labels.append(1 if label==2 else 0)
    test_texts.append(text)

print(train_texts[0])
print(train_labels[0])
print(test_texts[0])
print(test_labels[0])

I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.
1
This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and purple too. What was up with that? <br /><br />Truly a horri

## 학습 및 검증 데이터 분리
학습데이터 800개와 테스트 검증데이터 200개

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels,  val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=3)
print(len(train_texts))
print(len(train_labels))
print(len(val_texts))
print(len(val_labels))

800
800
200
200


## 토크나이징 및 인코딩
사전 학습된 distilbert-based-uncased 모델에 투입하기 위해 토크나이저를 사용하여 인코딩

토큰은 입력 텍스트의 기본 구성요소를 의미하며, 일반적으로 단어를 지칭합니다.
토크나이징은 입력 텍스트를 단어로 구분해 내는 과정입니다.

토크나이징 결과를 딥러닝 모델이 사용하기 위해서는 잘게 구분된 토큰(텍스트 형태)을 숫자로 전환해야 합니다.
이 전환과정을 인코딩이라고 합니다.

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/mnt/d/workspace/colab_try/.venv/lib/python3.

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

print(train_encodings["input_ids"][0][:5])
print(tokenizer.decode(train_encodings["input_ids"][0][:5]))

[101, 4937, 11350, 2038, 2048]
[CLS] cat soup has two


# 문제

데이터세트 클래스 생성

문제: Torch.utils.data.Dataset을 상속하는 IMDbDataset 라는 클래스를 스스로 작성하세요.

그리고 문제 007의 IMDb 데이터세트에서 생성한 학습 (Train_encodings), 검증(val_encodings), 테스트 데이터세트(test_encodings)를 입력해서

이 클래스를 인스턴스화합니다. 클래스 및 인스턴스화 개념은 문제 002를 참고하세요. 이를 데이터세트 후속 문제에서 파인튜닝에 사용합니다.

In [ ]:
import torch
class IMDbDataset(DistilBertTokenizerFast):
    
    def __init__(self, encodings, labels):
        """생성자

        Args:
            encodings (_type_): 초기화
            labels (_type_): 초기화
        """        
        self.encodings = encodings
        self.labels = labels


    def __getitem__(self, idx: int):
        """아이템 반환 매서드

        Args:
            idx (integer): index

        Returns:
            _type_: _description_
        """        
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

# 반복 가능 타입, 딕셔너리
for i in train_dataset:
    print(i)
    break

{'input_ids': tensor([  101,  4937, 11350,  2038,  2048,  1000,  7592, 14433,  1000,  1011,
         2828, 18401,  2015, 28866,  2075,  2006,  1037, 13576,  4440,  2083,
         1996, 25115,  1010,  2073,  2505,  2064,  4148,  1010,  1998,  2515,
         1012,  2023,  2568,  1011,  4440,  4691,  4004,  2460,  3594,  2053,
        13764,  8649,  1010,  4942, 21532,  2773, 22163,  2612,  1012,  2045,
         2003,  2053,  2126,  1997,  7851,  2023, 17183, 14088,  9476,  3272,
         2000,  2425,  2017,  2000,  2156,  2009,  2005,  4426,  1012,  1998,
         2191,  2469,  2053,  2028,  2104,  2184,  2003,  1999,  1996,  2282,
         1012,  4487,  6491,  6633,  5677,  3672,  1998,  2064,  3490, 10264,
         2964,  1998, 18186,  1998,  9576,  2854,  1998,  5573,  2331,  1998,
         2655,  3560, 27770,  2005,  2500,  2024,  2691,  6991,  1012,  7481,
         1012,  3383,  1996,  2087, 13432,  3746,  2003,  2008,  1997,  2019,
        10777,  3605,  1997,  2300,  2008,  1996, 

## 사전 학습 모델 불러오기.!

DistilBertForSequenceClassification 클래스를 사용해서 사전 학습모델을 불러오세연

In [ ]:
from transformers import DistilBertForSequenceClassification
from transformers import pipeline

pipeline.model(DistilBertForSequenceClassification)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


RuntimeError: Failed to import transformers.models.distilbert.modeling_distilbert because of the following error (look up to see its traceback):
No module named 'torch._custom_ops'